<a href="https://colab.research.google.com/github/pradeepram80/QA/blob/main/Basic_QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QA System

QA is a fun AI problem. Lets try it out on a few public documents using Haystack APIs


In [1]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git


  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-ddx8parh
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-ddx8parh


### Prepare environment

#### Colab: Enable the GPU runtime
Make sure you enable the GPU runtime to experience decent speed in this tutorial.
**Runtime -> Change Runtime type -> Hardware accelerator -> GPU**

In [2]:
# Make sure you have a GPU running
!nvidia-smi

Wed Sep 15 03:19:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


## Document Store

Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of `DocumentStore` include `ElasticsearchDocumentStore`, `FAISSDocumentStore`,  `SQLDocumentStore`, and `InMemoryDocumentStore`.

**Here:** We recommended Elasticsearch as it comes preloaded with features like [full-text queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector storage for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

**Alternatives:** If you are unable to setup an Elasticsearch instance, then follow the [Tutorial 3](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial3_Basic_QA_Pipeline_without_Elasticsearch.ipynb) for using SQL/InMemory document stores.

### Start an Elasticsearch server
You can start Elasticsearch on your local machine instance using Docker. If Docker is not readily available in your environment (eg., in Colab notebooks), then you can manually download and execute Elasticsearch from source.

In [4]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

09/15/2021 03:20:21 - INFO - haystack.utils -   Starting Elasticsearch ...
09/15/2021 03:20:21 - WARNING - haystack.utils -   Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [5]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [6]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

09/15/2021 03:21:10 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.094s]
09/15/2021 03:21:11 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.444s]
09/15/2021 03:21:11 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.288s]


## Preprocessing of documents

Haystack provides a customizable pipeline for:
 - converting files into texts
 - cleaning texts
 - splitting texts
 - writing them to a Document Store

In this tutorial, we download some public mu documents, apply a basic cleaning function, and index them in Elasticsearch.

In [10]:
#need pdftotext installed
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz && tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

# Let's first fetch some documents that we want to query
# Here: 27 pdfs from mu website searched using google
doc_dir = "mu_docs"

# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
dicts = convert_files_to_dicts(dir_path=doc_dir, split_paragraphs=True)

from haystack.preprocessor import PreProcessor
processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
#    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0
)
dicts = processor.process(dicts)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is:
# {
#    'text': "<DOCUMENT_TEXT_HERE>",
#    'meta': {'name': "<DOCUMENT_NAME_HERE>", ...}
#}
# (Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and
# can be accessed later for filtering or shown in the responses of the Pipeline)

# Let's have a look at the first 3 entries:
print(dicts[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

--2021-09-15 03:29:11--  https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23024720 (22M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.03.tar.gz.3’

xpdf-tools-linux-4. 100%[===================>]  21.96M  37.0MB/s    in 0.6s    

2021-09-15 03:29:12 (37.0 MB/s) - ‘xpdf-tools-linux-4.03.tar.gz.3’ saved [23024720/23024720]

xpdf-tools-linux-4.03/
xpdf-tools-linux-4.03/ANNOUNCE
xpdf-tools-linux-4.03/bin32/
xpdf-tools-linux-4.03/bin32/pdftotext
xpdf-tools-linux-4.03/bin32/pdfinfo
xpdf-tools-linux-4.03/bin32/pdftopng
xpdf-tools-linux-4.03/bin32/pdfimages
xpdf-tools-linux-4.03/bin32/pdftoppm
xpdf-tools-linux-4.03/bin32/pdftops
xpdf-tools-linux-4.03/bin32/pdfdetach
xpdf-tools-linux-4.03/bin32/pdffonts
xpdf-tools-linux-4.03/bin32/pdftohtml
xpdf-tools-linux-4.03/CHANGES
xpdf-tools-li

09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/Micron Achieves Comprehensive Global Pay Equity For All Underrepresented Groups.pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/Micron Launches Low-Power Memory Qualified for Automotive Safety Applications (1).pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/tn1342_mg_s29gl_np_to_mt28ew_automotive.pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/32gb_ddr4_x4x8_2cs_twindie.pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/Micron Appoints Linnie Haynesworth to Its Board of Directors.pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/Micron Technology Announces Upcoming Investor Events (2).pdf
09/15/2021 03:29:13 - INFO - haystack.preprocessor.utils -   Converting mu_docs/tn1252_sw_drivers_M58WR0_64Mb.pdf
09/15/2021 03:29:13 - INFO - haysta

[{'text': 'Micron Achieves Comprehensive Global Pay Equity For All Underrepresented Groups\nMarch 23, 2021\nPay gap eliminated in total employee compensation across base pay, bonuses and stock rewards\nBOISE, Idaho, March 23, 2021 (GLOBE NEWSWIRE) -- Micron Technology, Inc. (Nasdaq: MU), today announced that it has reached comprehensive global pay equity in total employee compensation across base, bonuses and stock rewards. Last year, the company set a goal to achieve pay equity for all employees as a key pillar of its diversity, equality and inclusion initiative. In line with the company\'s commitment to drive equal pay, inclusive benefits and fairness in its people practices, this new milestone reinforces Micron\'s focus and commitment to create an environment where every team member is valued and respected. "Providing fair and equal pay for all team members is key to driving employee wellbeing, passion and confidence that is critical to the success of Micron\'s business," said April

09/15/2021 03:29:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.708s]
09/15/2021 03:29:18 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
09/15/2021 03:29:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.031s]
09/15/2021 03:29:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.016s]
09/15/2021 03:29:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.029s]
09/15/2021 03:29:23 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.031s]
09/15/2021 03:29:24 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
09/15/2021 03:29:25 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.010s]


## Initalize Retriever, Reader,  & Pipeline

### Retriever

Retrievers help narrowing down the scope for the Reader to smaller units of text where a given question could be answered.
They use some simple but fast algorithm.

**Here:** We use Elasticsearch's default BM25 algorithm

**Alternatives:**

- Customize the `ElasticsearchRetriever`with custom queries (e.g. boosting) and filters
- Use `TfidfRetriever` in combination with a SQL or InMemory Document store for simple prototyping and debugging
- Use `EmbeddingRetriever` to find candidate documents based on the similarity of embeddings (e.g. created via Sentence-BERT)
- Use `DensePassageRetriever` to use different embedding models for passage and query (see Tutorial 6)

In [11]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [12]:
# Alternative: An in-memory TfidfRetriever based on Pandas dataframes for building quick-prototypes with SQLite document store.

# from haystack.retriever.sparse import TfidfRetriever
# retriever = TfidfRetriever(document_store=document_store)

### Reader

A Reader scans the texts returned by retrievers in detail and extracts the k best answers. They are based
on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers.
With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

**Here:** a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

**Alternatives (Reader):** TransformersReader (leveraging the `pipeline` of the Transformers package)

**Alternatives (Models):** e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)

**Hint:** You can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"

#### FARMReader

In [13]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

09/15/2021 03:29:47 - INFO - farm.utils -   Using device: CUDA 
09/15/2021 03:29:47 - INFO - farm.utils -   Number of GPUs: 1
09/15/2021 03:29:47 - INFO - farm.utils -   Distributed Training: False
09/15/2021 03:29:47 - INFO - farm.utils -   Automatic Mixed Precision: None
09/15/2021 03:29:47 - INFO - filelock -   Lock 139680513442448 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

09/15/2021 03:29:47 - INFO - filelock -   Lock 139680513442448 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock
09/15/2021 03:29:48 - INFO - filelock -   Lock 139680519977296 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

09/15/2021 03:30:06 - INFO - filelock -   Lock 139680519977296 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock
Some weights of the model checkpoint at deepset/roberta-base-squad2 were not used when initializing RobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['ro

...

09/15/2021 03:30:14 - INFO - filelock -   Lock 139680524500112 released on /root/.cache/huggingface/transformers/b87d46371731376b11768b7839b1a5938a4f77d6bd2d9b683f167df0026af432.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
09/15/2021 03:30:14 - INFO - filelock -   Lock 139680513949584 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

09/15/2021 03:30:15 - INFO - filelock -   Lock 139680513949584 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
09/15/2021 03:30:15 - INFO - filelock -   Lock 139680513949584 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

09/15/2021 03:30:15 - INFO - filelock -   Lock 139680513949584 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
09/15/2021 03:30:15 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
09/15/2021 03:30:15 - INFO - farm.utils -   Using device: CUDA 
09/15/2021 03:30:15 - INFO - farm.utils -   Number of GPUs: 1
09/15/2021 03:30:15 - INFO - farm.utils -   Distributed Training: False
09/15/2021 03:30:15 - INFO - farm.utils -   Automatic Mixed Precision: None
09/15/2021 03:30:15 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
09/15/2021 03:30:15 - INFO - farm.infer -    0    0 
09/15/2021 03:30:15 - INFO - farm.infer -   /w\  /w\
09/15/2021 03:30:15 - INFO - farm.infer -   /'\  / \
09/15/2021 03:30:15 - INFO - farm.infer -     
09/15/2021 03:30:15 - INFO - farm.utils -   Usi

#### TransformersReader

In [ ]:
# Alternative:
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `ExtractiveQAPipeline` that combines a retriever and a reader to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).

In [15]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

## Voilà! Ask a question!

In [33]:
# You can configure how many candidates the reader and retriever shall return
# The higher the top_k, the better (but also the slower) your answers.
prediction = pipe.run(
    query="what is Linnie Haynesworth's role in Micron?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print ()
print_answers(prediction)

09/15/2021 03:52:36 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.019s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 19.08 Batches/s]



{   'answers': [   {   'answer': 'board of directors',
                       'context': 'ons, today announced the appointment of '
                                  'Linnie Haynesworth to its board of '
                                  'directors. Haynesworth has more than 30 '
                                  'years of experience in technology, ',
                       'document_id': '51f53d6de3b4d5717db1757b44026092',
                       'meta': {   '_split_id': 0,
                                   'name': 'Micron Appoints Linnie Haynesworth '
                                           'to Its Board of Directors.pdf'},
                       'offset_end': 84,
                       'offset_end_in_doc': 228,
                       'offset_start': 66,
                       'offset_start_in_doc': 210,
                       'score': 0.8297300338745117},
                   {   'answer': 'Board of Directors',
                       'context': 'Micron Appoints Linnie Haynesworth 